![xframe](images/xframe.png)

<center> <h1>xframe is a dataframe for C++, based on xtensor and xtl</h1> </center>

In [1]:
#include <string>
#include <iostream>

#include "xtl/xbasic_fixed_string.hpp"

#include "xtensor/xio.hpp"
#include "xtensor/xrandom.hpp"
#include "xtensor/xmath.hpp"

#include "xframe/xio.hpp"
#include "xframe/xvariable.hpp"
#include "xframe/xvariable_view.hpp"
#include "xframe/xvariable_masked_view.hpp"
#include "xframe/xio.hpp"

In [2]:
using fstring = xtl::xfixed_string<55>;

using data_type = xt::xoptional_assembly<xt::xarray<double>, xt::xarray<bool>>;
using coordinate_type = xf::xcoordinate<fstring, data_type::size_type>;

using variable_type = xf::xvariable<coordinate_type, data_type>;

# Variable with 2D Data: Apparent Temperature

## Dry temperature variable

In [3]:
data_type dry_temperature_data = xt::eval(xt::random::rand({6, 3}, 15., 25.));
dry_temperature_data(0, 0).has_value() = false;
dry_temperature_data(2, 1).has_value() = false;

In [4]:
dry_temperature_data

N/A,23.3501,24.6887
17.2103,18.0817,20.4722
16.8838,N/A,24.9646
24.6769,22.2584,24.8111
16.0986,22.9811,17.9703
15.0478,16.1246,21.3976


In [5]:
auto time_axis = xf::axis({"2018-01-01", "2018-01-02", "2018-01-03", "2018-01-04", "2018-01-05", "2018-01-06"});

In [6]:
auto dry_temperature = variable_type(
    dry_temperature_data,
    {
        {"date", time_axis},
        {"city", xf::axis({"London", "Paris", "Brussels"})}
    }
);

In [7]:
dry_temperature

2018-01-01,N/A,23.3501,24.6887
2018-01-02,17.2103,18.0817,20.4722
2018-01-03,16.8838,N/A,24.9646
2018-01-04,24.6769,22.2584,24.8111
2018-01-05,16.0986,22.9811,17.9703
2018-01-06,15.0478,16.1246,21.3976


In [8]:
dry_temperature.select({{"city", "London"}, {"date", "2018-01-04"}})

24.6769

In [9]:
dry_temperature.locate("2018-01-03", "Brussels")

24.9646

In [10]:
dry_temperature.locate("2018-01-03", "Paris")

N/A

## Relative humidity variable

In [11]:
data_type relative_humidity_data = xt::eval(xt::random::rand({3}, 50.0, 70.0));

auto relative_humidity = variable_type(
    relative_humidity_data,
    {
        {"city", xf::axis({"Paris", "London", "Brussels"})}
    }
);

relative_humidity

Paris,67.5686
London,60.0733
Brussels,65.9586


## Compute water vapour pressure using Broadcasting

In [12]:
dry_temperature

2018-01-01,N/A,23.3501,24.6887
2018-01-02,17.2103,18.0817,20.4722
2018-01-03,16.8838,N/A,24.9646
2018-01-04,24.6769,22.2584,24.8111
2018-01-05,16.0986,22.9811,17.9703
2018-01-06,15.0478,16.1246,21.3976


In [13]:
auto water_vapour_pressure = 0.01 * relative_humidity * 6.1 * xt::exp((17.27 * dry_temperature) / (237.7 + dry_temperature));

In [14]:
water_vapour_pressure

2018-01-01,N/A,17.1746,20.4322
2018-01-02,13.2269,12.4228,15.8251
2018-01-03,12.9565,N/A,20.7708
2018-01-04,20.9164,16.077,20.5819
2018-01-05,12.326,16.7965,13.5448
2018-01-06,11.5244,10.9769,16.7499


## Wind speed

In [15]:
data_type wind_speed_data = xt::eval(xt::random::rand({6, 3}, 1., 2.));
wind_speed_data(2, 0).has_value() = false;

auto wind_speed = variable_type(
    wind_speed_data, 
    {
        {"date", time_axis},
        {"city", xf::axis({"Paris", "Brussels", "London"})}
    }
);

wind_speed

2018-01-01,1.36129,1.21192,1.68136
2018-01-02,1.39874,1.74065,1.47476
2018-01-03,N/A,1.17387,1.30191
2018-01-04,1.79728,1.31655,1.87243
2018-01-05,1.14911,1.99407,1.8219
2018-01-06,1.12518,1.76375,1.49059


## And compute apparent temperature

In [16]:
dry_temperature

2018-01-01,N/A,23.3501,24.6887
2018-01-02,17.2103,18.0817,20.4722
2018-01-03,16.8838,N/A,24.9646
2018-01-04,24.6769,22.2584,24.8111
2018-01-05,16.0986,22.9811,17.9703
2018-01-06,15.0478,16.1246,21.3976


In [17]:
auto apparent_temperature = dry_temperature + 0.348 * water_vapour_pressure - 0.7 * wind_speed + 0.7/(wind_speed + 10) - 4.25;

In [18]:
apparent_temperature

2018-01-01,N/A,24.2909,26.4321
2018-01-02,16.6456,16.996,20.758
2018-01-03,N/A,N/A,27.0935
2018-01-04,26.5071,22.7435,26.4718
2018-01-05,15.3965,23.2387,17.2178
2018-01-06,14.0836,14.5195,21.9941


## Variable with 3D data

In [19]:
data_type pressure_data = {{{ 1.,  2., 3. },
                            { 4.,  5., 6. },
                            { 7.,  8., 9. }},
                           {{ 1.3, 1.5, 1.},
                            { 2., 2.3, 2.4},
                            { 3.1, 3.8, 3.}},
                           {{ 8.5, 8.2, 8.6},
                            { 7.5, 8.6, 9.7},
                            { 4.5, 4.4, 4.3}}};

In [20]:
auto pressure = variable_type(
    pressure_data,
    {
        {"x", xf::axis(3)},
        {"y", xf::axis(3, 6, 1)},
        {"z", xf::axis(3)},
    }
);

In [21]:
pressure

0 1 2 0 3 1 2 3 4 4 5 6 5 7 8 9 1 3 1.3 1.5 1 4 2 2.3 2.4 5 3.1 3.8 3 2 3 8.5 8.2 8.6 4 7.5 8.6 9.7 5 4.5 4.4 4.3

In [22]:
pressure.select({{"x", 1}, {"y", 4}, {"z", 1}})

2.3

In [23]:
pressure.locate(0, 5, 2)

9

## Masking

In [24]:
auto masked_pressure = xf::where(
    pressure,
    not_equal(pressure.axis<int>("x"), 2) && pressure.axis<int>("z") < 2
);

In [25]:
masked_pressure

0 1 2 0 3 1 2 N/A 4 4 5 N/A 5 7 8 N/A 1 3 1.3 1.5 N/A 4 2 2.3 N/A 5 3.1 3.8 N/A 2 3 N/A N/A N/A 4 N/A N/A N/A 5 N/A N/A N/A